In [1]:
%reload_ext ishbook
import pandas as pd
pd.options.mode.chained_assignment = None
import plus
from nltk.tokenize import word_tokenize
import nltk
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# v1 Refund Report:

* Total volume of Refund Requests sent by SMB floor coaches to Finance
* Total completed Refund Amounts
* Raw data including advid, refunded amount, requestor (floor coach name)
* Volume of Accounts with a reassignment within 30 days of the refund being completed – if we find there is a significant volume of accounts changing hands, then we can look to include rep assignments.



*Methodology*

1. First find Credit/Refund Requests sent by SMB floor coaches using the 'Credit/Refund Request' category
2. Indicate which requests were Refunds, Credits or Refund & Credit requests (since we obviously wouldn't show a refunded amount for requests that are only credits) -- we can do this by creating a function and mapping it to the request note
3. Use plus to get the account_ids and insert a stringed list into the deathstar index in order to look for refunded accounts and amounts
4. Use plus to also obtain current account status and HAJ status info as well as current rep assignments

In [2]:
%%ish
requests = from repaccountnotes 2016-09-01 2016-09-28 where username IN (jdrosos,virginia,jcrocco,talia) category_name = 'Credit/Refund Request' group by username,advid,unixtime,note

output = None

In [3]:
requests_info = plus.get_advertiser_info(requests.advid,intype='advid')
requests_info.reset_index(inplace=True)


LDAP name:
abrar
Password:
········


In [4]:
requests['test']=requests.note.apply(word_tokenize)

In [5]:
requests.head(1)

,username,advid,unixtime,note,,test
0,virginia,8186357,1473708226,$26.92 Type of request: CreditAmt of Request:...,1,"[$, 26.92, Type, of, request, :, CreditAmt, of..."


In [6]:
writer = pd.ExcelWriter('check.xlsx', engine='xlsxwriter')
requests.to_excel(writer,sheet_name = 'contact')

In [7]:
# Create a regex

credits = "[cC]redit"
refunds = "[rR]efund|$R|R|$R|R:|"
both = "[cC]redit [rR]efund"

In [8]:
# Create function to specify refunds or credits

def refund(string_in):
    if 'credit refund' in string_in:
        return 'Both'
    elif 'Refund/Credit' in string_in:
        return 'Both'
    elif 'Credit' in string_in:
        return 'Credit'
    elif 'credit' in string_in:
        return 'Credit'
    elif 'refund' in string_in:
        return 'Refund'
    elif 'Refund' in string_in:
        return 'Refund'
    elif '$Refund' in string_in:
        return 'Refund'
    else:
        return 'Check'
    

In [9]:
# Apply the function and create a new column

requests['type'] = requests.note.map(refund).copy()

In [10]:
requests

,username,advid,unixtime,note,,test,type
0,virginia,8186357,1473708226,$26.92 Type of request: CreditAmt of Request:...,1,"[$, 26.92, Type, of, request, :, CreditAmt, of...",Credit
1,virginia,5509632,1474406277,$80.44Type of request: RefundAmt of Request: ...,1,"[$, 80.44Type, of, request, :, RefundAmt, of, ...",Refund
2,virginia,8659946,1473686934,C$111.43Type of request: CreditAmt of Request...,1,"[C, $, 111.43Type, of, request, :, CreditAmt, ...",Credit
3,virginia,1912394,1474915788,C$202.31Type of request: CreditAmt of Request...,1,"[C, $, 202.31Type, of, request, :, CreditAmt, ...",Credit
4,virginia,8239814,1473256325,C$444.72Type of request: Credit Amt of Reques...,1,"[C, $, 444.72Type, of, request, :, Credit, Amt...",Credit
5,virginia,7744905,1473256011,C$74.30 Type of request: Credit Amt of Reques...,1,"[C, $, 74.30, Type, of, request, :, Credit, Am...",Credit
6,virginia,3627018,1474652690,$ R $378.79Type of Request: RefundAmount of Re...,1,"[$, R, $, 378.79Type, of, Request, :, RefundAm...",Refund
7,talia,8769908,1474897559,"$1,456.80 / $265.48Type of request: Refund/Cre...",1,"[$, 1,456.80, /, $, 265.48Type, of, request, :...",Both
8,virginia,8759232,1473364457,$C $80.81Type of Request: CreditAmount of Requ...,1,"[$, C, $, 80.81Type, of, Request, :, CreditAmo...",Credit
9,virginia,4575678,1474982551,$R $27.22 $C 98.87Refund: $27.22 Credit $98.87...,1,"[$, R, $, 27.22, $, C, 98.87Refund, :, $, 27.2...",Credit


In [11]:
# String together accountids to insert into deathstar IQL query

acctids = ','.join(str(x) for x in requests_info.account_id)

In [12]:
%%ish

refunds = from deathstar 2016-01-01 today where accountid IN ({acctids}) requesttype = refundprofile group by accountid/*account_id*/ select amount/100/*refund_amount*/
output = None

In [13]:
# Convert unixtimes to datetimes

requests['request_date']=pd.to_datetime(requests.unixtime,unit='s')

In [14]:
# Merge request info with account id info with refund info

refund_requests = requests.merge(requests_info,how='left',on='advid').merge(refunds,how='left',on='account_id')

In [15]:
# Narrow down to categories I care about

Refunds = refund_requests[['advid','account_id','username','request_date','active','showHostedJobs','service_rep','sales_rep','refund_amount','type_x','note']]

In [16]:
# Replace 'active' column (account status) with understandable terms

Refunds.active.replace({0:'Disabled',1:'Approved',2:'Awaiting Authorization'},inplace=True)

In [17]:
# Get back service and sales rep assignments

service_rep = plus.get_user_info(Refunds.service_rep.unique())
service_rep.reset_index(inplace=True)

sales_rep = plus.get_user_info(Refunds.sales_rep.unique())
sales_rep.reset_index(inplace=True)

In [18]:
# Merge service and sales rep dataframes together

refund_reps = Refunds.merge(service_rep,how='left',left_on=['service_rep'],right_on=['index']).merge(sales_rep,how='left',left_on=['sales_rep'],right_on=['index'])

In [19]:
# Once more, narrow down to columns I care about

Final = refund_reps[['advid','username_x','request_date','refund_amount','type_x','active_x','showHostedJobs','username_y','username']]

# Rename columns

Final.columns=['Advertiser ID','SMB Floor Coach','Request Date','Refunded Amount','Request Type','Current Account Status','Current HAJ Status','Current Service Rep','Current Sales Rep']

In [20]:
# Replace NAs with 0

Final['Refunded Amount'].fillna(0,inplace=True)

In [21]:
# Remove duplicates

Final.drop_duplicates(inplace=True)
Final.head(2)

,Advertiser ID,SMB Floor Coach,Request Date,Refunded Amount,Request Type,Current Account Status,Current HAJ Status,Current Service Rep,Current Sales Rep
0,8186357,virginia,2016-09-12 19:23:46,0.0,Credit,1,verified,CAsharedsupport,lchalmers
1,5509632,virginia,2016-09-20 21:17:57,0.0,Refund,1,verified,smbsupp,na


In [22]:
Final.to_csv('Refundv1.csv')

In [23]:
new = pd.read_csv('refund.csv')
new.head(2)

,Advid,Date,Type,Refund $,Credit $,Free Click $,Reason,Notes,Initials
0,NaN,9/1/2016,Refund,$459.11,NaN,NaN,Bid Opt Overspend,NaN,CL
1,NaN,9/1/2016,Refund/Credit,$500.40,$94.29,NaN,Daily Budget - Free,Unintended budgets on ads- started spending wh...,CG


In [24]:
new['Advid'].fillna(0, inplace=True)

In [25]:
new['Advid'] = new['Advid'].astype(int)

In [26]:
new1 = new.rename(columns={'Advid': 'Advertiser ID'})

In [27]:
new1.head(2)

,Advertiser ID,Date,Type,Refund $,Credit $,Free Click $,Reason,Notes,Initials
0,0,9/1/2016,Refund,$459.11,NaN,NaN,Bid Opt Overspend,NaN,CL
1,0,9/1/2016,Refund/Credit,$500.40,$94.29,NaN,Daily Budget - Free,Unintended budgets on ads- started spending wh...,CG


In [28]:
finaldf = Final.merge(new1, on = 'Advertiser ID', how = 'left')
finaldf.head(2)

,Advertiser ID,SMB Floor Coach,Request Date,Refunded Amount,Request Type,Current Account Status,Current HAJ Status,Current Service Rep,Current Sales Rep,Date,Type,Refund $,Credit $,Free Click $,Reason,Notes,Initials
0,8186357,virginia,2016-09-12 19:23:46,0.0,Credit,1,verified,CAsharedsupport,lchalmers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5509632,virginia,2016-09-20 21:17:57,0.0,Refund,1,verified,smbsupp,na,9/20/2016,Refund,$80.44,NaN,NaN,Recurring Budget,NaN,VF


In [29]:
finaldf = finaldf[['Advertiser ID','SMB Floor Coach', 'Request Date', 'Refunded Amount', 'Request Type', 'Current Account Status', 'Current HAJ Status', 'Current Service Rep', 'Reason ']]


In [30]:
finaldf.to_csv('refundnew.csv')

# Reason


In [31]:
leave = "new owner.*|.*account owner.*|.*new contact.*|.*former employee.*|.*contact change.*|.*left company.*"
openCC = "old jobs.*|.*open jobs.*|.*resume database.*|.*old job.*"
salesrep = "sales rep.*|.*sales.*|.*AE.*|.*account executive.*|.*account executive/.*|.*/sales.*|.*/Sales.*|.*Account Executive.*"
overspend = "over spend.*|.*overspend.*|.*Recurring budget.*|.*recurring budget.*|.*over budget.*|.*spent over.*|.*spend over.*"
clienterror = "accident.*|.*daily.*|.*no idea.*|.*thought.*|confused|.*forgot.*|.*unaware.*|.*misunderstood.*|intended|Intended|.*lack of education.*"
hacked = "hacked.*|.*Hacked.*|.*hacking.*|.*Hacking.*"
salaryfield = "salary field.*|.*Salary Field.*|.*Salary field.*"

In [32]:
# Create function to specify refunds or credits

def note1(string_in):
    if 'leave' in string_in:
        return 'Account Owner Changed'
    elif 'openCC' in string_in:
        return 'Added CC with Open Jobs'
    elif 'sales rep' in string_in:
        return 'AE'
    elif 'overspend' in string_in:
        return 'Budget Overspend'
    elif 'clienterror' in string_in:
        return 'Client Error'
    elif 'hacked' in string_in:
        return 'Account Hacked'
    elif 'salaryfield' in string_in:
        return 'Salary Field'
    else:
        return 'Other'

In [33]:
requests['reason'] = requests.note.map(note1).copy()

In [34]:
requests

,username,advid,unixtime,note,,test,type,request_date,reason
0,virginia,8186357,1473708226,$26.92 Type of request: CreditAmt of Request:...,1,"[$, 26.92, Type, of, request, :, CreditAmt, of...",Credit,2016-09-12 19:23:46,Other
1,virginia,5509632,1474406277,$80.44Type of request: RefundAmt of Request: ...,1,"[$, 80.44Type, of, request, :, RefundAmt, of, ...",Refund,2016-09-20 21:17:57,Other
2,virginia,8659946,1473686934,C$111.43Type of request: CreditAmt of Request...,1,"[C, $, 111.43Type, of, request, :, CreditAmt, ...",Credit,2016-09-12 13:28:54,Other
3,virginia,1912394,1474915788,C$202.31Type of request: CreditAmt of Request...,1,"[C, $, 202.31Type, of, request, :, CreditAmt, ...",Credit,2016-09-26 18:49:48,Other
4,virginia,8239814,1473256325,C$444.72Type of request: Credit Amt of Reques...,1,"[C, $, 444.72Type, of, request, :, Credit, Amt...",Credit,2016-09-07 13:52:05,Other
5,virginia,7744905,1473256011,C$74.30 Type of request: Credit Amt of Reques...,1,"[C, $, 74.30, Type, of, request, :, Credit, Am...",Credit,2016-09-07 13:46:51,Other
6,virginia,3627018,1474652690,$ R $378.79Type of Request: RefundAmount of Re...,1,"[$, R, $, 378.79Type, of, Request, :, RefundAm...",Refund,2016-09-23 17:44:50,Other
7,talia,8769908,1474897559,"$1,456.80 / $265.48Type of request: Refund/Cre...",1,"[$, 1,456.80, /, $, 265.48Type, of, request, :...",Both,2016-09-26 13:45:59,Other
8,virginia,8759232,1473364457,$C $80.81Type of Request: CreditAmount of Requ...,1,"[$, C, $, 80.81Type, of, Request, :, CreditAmo...",Credit,2016-09-08 19:54:17,Other
9,virginia,4575678,1474982551,$R $27.22 $C 98.87Refund: $27.22 Credit $98.87...,1,"[$, R, $, 27.22, $, C, 98.87Refund, :, $, 27.2...",Credit,2016-09-27 13:22:31,Other


In [35]:
writer = pd.ExcelWriter('check1.xlsx', engine='xlsxwriter')
requests.to_excel(writer,sheet_name = 'idc2')
writer.save()